In [2]:
import glob
import argparse
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import oggm
from oggm import utils

%matplotlib qt 

In [3]:
# Setup oggm: important we want to use oggm's version 62 of all glacier geometries. This command will download them.
directory = utils.get_rgi_dir(version='62')
directory

'C:\\Users\\luisa\\OGGM\\rgi\\RGIV62'

In [4]:
path_O1_shp = directory + r"\00_rgi62_regions\00_rgi62_O1Regions.shp" # shp file of regional boundaries

# Import 20-bin gridded training dataset
path_metadata = "Beginning"
metadata_file = r"\metadata19_hmineq0.0_tmin20050000_mean_grid_20.csv"
glathida_rgis = pd.read_csv(path_metadata + metadata_file, low_memory=False)

In [5]:
# See all RGI's regional boundaries
# See pag 8 at https://nsidc.org/sites/nsidc.org/files/technical-references/RGI_Tech_Report_V6.0.pdf
world = gpd.read_file(path_O1_shp)      # import as geopandas dataframe
print(world)
fig, ax = plt.subplots()
for n, rgi_series in enumerate(world['geometry']):
    x, y = rgi_series.exterior.xy
    min_x, max_x = min(x), max(x)
    min_y, max_y = min(y), max(y)
    ax.plot(x, y, c='g')
    ax.text(np.mean([min_x, max_x]), np.mean([min_y, max_y]), f"{world['RGI_CODE'].iloc[n]}", color='green', fontsize=9)
plt.show()

                     FULL_NAME RGI_CODE WGMS_CODE  \
0                       Alaska        1       ALA   
1                       Alaska        1       ALA   
2       Western Canada and USA        2       WNA   
3         Arctic Canada, North        3       ACN   
4         Arctic Canada, South        4       ACS   
5          Greenland Periphery        5       GRL   
6                      Iceland        6       ISL   
7       Svalbard and Jan Mayen        7       SJM   
8                  Scandinavia        8       SCA   
9               Russian Arctic        9       RUA   
10                 Asia, North       10       ASN   
11                 Asia, North       10       ASN   
12              Central Europe       11       CEU   
13    Caucasus and Middle East       12       CAU   
14               Asia, Central       13       ASC   
15            Asia, South West       14       ASW   
16            Asia, South East       15       ASE   
17               Low Latitudes       16       

In [6]:
# Let's select one region of interest, e.g. rgi=11 (Central Europe)

# Dataframe of glaciers for rgi=11. This dataset contains a lot of info, some of them are also used to create the
# training dataset. RGIId is the official name of glaciers according to the Randolph Glacier Inventory. 
# In the column geometry you get the geometries. 

rgi = 11
oggm_rgi_shp = glob.glob(f"C:/Users/luisa/OGGM/rgi/RGIV62/{rgi}*/{rgi}*.shp")[0] # .shp file for rgi 11
oggm_rgi_glaciers = gpd.read_file(oggm_rgi_shp)                         # dataframe for rgi 11

In [11]:
# Plot one specific glacier by passing the RGIId code.
# Example: RGI60-11.01450 is the Aletsch Glacier, the biggest in Central Europe (rgi=11).
# You under the geometry column you find the glacier external boundary (.external)
# and the glacier nunataks, if any (.interiors). Nunataks are the inner glacier portions that are deglaciated (=rock).
# We can also find the points in the training dataset for this glacier (not all glaciers contain measurements)

glacier_geometry = oggm_rgi_glaciers.loc[oggm_rgi_glaciers['RGIId']=='RGI60-11.01450']['geometry'].item()

# Get the measurements in the training dataset by passing the glacier id
glathida_rgis_aletsch = glathida_rgis.loc[glathida_rgis['RGIId']=='RGI60-11.01450']
print(f"We have {len(glathida_rgis_aletsch)} points in the training dataset")

fig, ax = plt.subplots()
exterior_ring = glacier_geometry.exterior # External geometry
ax.plot(*exterior_ring.xy, c='b')
glacier_nunataks_list = [nunatak for nunatak in glacier_geometry.interiors] # Nunataks (list may be empty if no nunataks)
for nunatak in glacier_nunataks_list:
    ax.plot(*nunatak.xy, c='k', lw=0.8)
ax.scatter(x=glathida_rgis_aletsch['POINT_LON'], y=glathida_rgis_aletsch['POINT_LAT'], s=20, c='orange', label='Training points')
ax.legend()
plt.show()

We have 49 points in the training dataset


Plot the Glaciers with a different color then their surroundings for CNN.

In [9]:
color_glacier_margin = 'white'
color_glacier_out = 'grey'
color_glacier_in = 'white'
color_nunatak_margin = 'black'
color_nunatak = 'black'

In [12]:
fig, ax = plt.subplots()

# plot the exterior ring = glacier margin
exterior_ring = glacier_geometry.exterior 
ax.plot(*exterior_ring.xy, c=color_glacier_margin)

# plot the nunataks 
glacier_nunataks_list = [nunatak for nunatak in glacier_geometry.interiors]
for nunatak in glacier_nunataks_list:
    ax.plot(*nunatak.xy, c=color_nunatak_margin, lw=0.8)

bounds = glacier_geometry.bounds
x_min, y_min, x_max, y_max = bounds # get the boundaries for the plot

# add a margin to the bounding box so that we have it similar to the previous plot of the glacier - means boundary glacier != boundary plot
margin = 0.05 * max(x_max - x_min, y_max - y_min)
x_min -= margin
x_max += margin
y_min -= margin
y_max += margin
ax.set_xlim(x_min, x_max)
ax.set_ylim(y_min, y_max)

# create a large rectangle for the whole plot area
outer_rect = np.array([[x_min, y_min], [x_min, y_max], [x_max, y_max], [x_max, y_min], [x_min, y_min]])

# shade the area outside the glacier's exterior ring
ax.fill(*outer_rect.T, facecolor=color_glacier_out, edgecolor='none', alpha=0.5) # * -> it unpacks the tuple/array into separate arguments 
ax.fill(*exterior_ring.xy, facecolor=color_glacier_in, edgecolor='none')

# shade the interior of the nunataks 
for nunatak in glacier_nunataks_list:
    ax.fill(*nunatak.xy, facecolor=color_nunatak, edgecolor='none')

plt.show()

Questions: 
- edgecoler=None: is this correct or do we loose pixel?
- Should we plot the contour of the glacier? Loose pixel?
- Should we plot the contour of the nunataks? Loose pixel?

In [ ]:
# Plot the whole regional set of glaciers and all nunataks
# zoom in with matplotlib in interactive mode (%matplotlib qt ) to have a sense of all glacier and their nunataks

region11 = world.loc[12] # Regional boundary
fig, ax = plt.subplots()
for glacier_rgiid, glacier_geometry in zip(oggm_rgi_glaciers['RGIId'], oggm_rgi_glaciers['geometry']):

    exterior_ring = glacier_geometry.exterior
    glacier_nunataks_list = [nunatak for nunatak in glacier_geometry.interiors]

    ax.plot(*exterior_ring.xy, c='b')
    for nunatak in glacier_nunataks_list:
        ax.plot(*nunatak.xy, c='k', lw=0.8) # Plot glacier nunataks (if any)
ax.plot(*region11['geometry'].exterior.xy, c='g') # Regional boundary
ax.text(min(*region11['geometry'].exterior.xy[0])+1,
        max(*region11['geometry'].exterior.xy[1])-1, f'rgi {rgi}', c='g', fontsize=12)
plt.show()